In [3]:
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [4]:
filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/May2021/genetic_effect/scaled_absolute_eff_sizes_20mofas_as_spatial_downsample_50pct_cluster_labels.csv"
df_clusters = read.csv(filename, row.names = 1)

In [5]:
head(df_clusters[,c("label","gene")],2)

,label,gene
0,2,ABHD10_-3_111697439_T_G
1,9,ABHD12B_-14_51328222_C_T


In [6]:
# load cluster summaries
cluster_summaries = read.csv("/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/May2021/genetic_effect/scaled_absolute_eff_sizes_20mofas_as_spatial_downsample_50pct_cluster_summaries.csv", row.names=1)
colnames(cluster_summaries) = paste0("cluster",0:(ncol(cluster_summaries)-1))
head(cluster_summaries,2)

,cluster0,cluster1,cluster2,cluster3,cluster4,cluster5,cluster6,cluster7,cluster8,cluster9,cluster10,cluster11
HPSI0114i-bezi_3--DA--d30--0,0.4492843,-0.7952382,-0.4816106,0.8658570,-0.1223015,0.0950069,1.109847,0.4850352,-0.16113688,-0.4720093,0.2110775,-0.5376197
HPSI0114i-bezi_3--DA--d30--1,0.5092915,-0.8019595,-0.5017370,0.9154453,0.1000676,0.2789833,1.170581,0.6114963,-0.07019794,-0.5283503,0.2034086,-0.6379603


In [7]:
###################################################### 
### approach 1: enrichment of genes (eQTL) in the clusters

In [8]:
## load individual eQTL genetic effects
df = read.csv("/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/May2021/genetic_effect/MOFA10/eqtl_tot_genetic_effect_all.txt", sep="\t",row.names = 1)
colnames(df) = gsub("\\.","-",colnames(df))
head(df,2)
nrow(df)

,ABHD10_-3_111697439_T_G,ABHD12B_-14_51328222_C_T,ACKR2_-3_42817129_C_T,ADD2_-2_70894426_A_T,ADSL_-22_40757228_A_C,AIM1L_-1_26673825_C_T,AKAP7_-6_131691964_C_A,ALDH7A1_-5_126011412_G_A,ANKRD18B_-9_33524775_C_A,APIP_-11_34905987_T_C,...,YWHAE_-17_1246420_G_A,ZBBX_-3_167080934_T_G,ZC3HAV1_-7_138730361_T_C,ZFR2_-19_3840157_C_T,ZG16B_-16_2871121_T_C,ZNF423_-16_49599995_C_G,ZNF552_-19_58315273_G_A,ZNF738_-19_21474173_G_A,ZNF77_-19_2936535_G_A,ZP3_-7_76177284_C_T
HPSI0714i-iudw_1--DA--d30--0,-0.2108622,0.003924475,0.02206444,0.02834211,0.2235163,-0.03575032,-0.09473256,0.006901295,0.0006737957,-0.07499119,...,0.06962421,0.1120253,-0.04172133,0.01126897,0.1057579,-0.03101814,0.0909417,0.3020245,-0.1583842,-0.05596878
HPSI0714i-iudw_1--DA--d30--1,-0.2283969,-0.047217741,0.10596178,0.02514689,-0.1921801,-0.01471768,0.01878961,-0.171038975,-0.1105473675,-0.18686595,...,-0.21070417,0.3759854,0.08774666,0.02505916,-0.1950055,-0.22780547,0.0775938,0.3452492,-0.3472152,-0.15259400


[1] 8352

In [9]:
##########################################################################################
### approach 2: enrichment of genes correlated with each clusters (using the cluster summaries)

In [10]:
input_files_dir = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/May2021/input_files/"
# Phenotype
phenotype_file = paste0(input_files_dir,"phenotype.csv")

In [11]:
df0 = read.csv(phenotype_file, row.names=1)
newcols = gsub("\\.","-",colnames(df0))
# head(newcols)
colnames(df0) = newcols
df0[1:5,1:5]

,HPSI0714i-iudw_1--DA--d30--0,HPSI0714i-iudw_1--DA--d30--1,HPSI0714i-iudw_1--DA--d30--2,HPSI0714i-iudw_1--DA--d30--3,HPSI0714i-iudw_1--DA--d30--4
MIR1302.10,0,0,0,0,0
FAM138A,0,0,0,0,0
OR4F5,0,0,0,0,0
RP11.34P13.7,0,0,0,0,0
RP11.34P13.8,0,0,0,0,0


In [12]:
df1 = df0[,rownames(cluster_summaries)]
ncol(df0)
ncol(df1)
df1[1:5,1:5]

[1] 9219

[1] 8352

,HPSI0114i-bezi_3--DA--d30--0,HPSI0114i-bezi_3--DA--d30--1,HPSI0114i-bezi_3--DA--d30--10,HPSI0114i-bezi_3--DA--d30--11,HPSI0114i-bezi_3--DA--d30--12
MIR1302.10,0,0,0,0,0
FAM138A,0,0,0,0,0
OR4F5,0,0,0,0,0
RP11.34P13.7,0,0,0,0,0
RP11.34P13.8,0,0,0,0,0


In [13]:
rowsums = rowSums(df1)
head(rowsums)

MIR1302.10      FAM138A        OR4F5 RP11.34P13.7 RP11.34P13.8   AL627309.1 
   33.089211     0.000000     0.000000   154.172128     8.009869   464.634623

In [14]:
# remove all 0 genes
zero_genes = names(rowsums[rowsums==0])
head(zero_genes)
length(zero_genes)

[1] "FAM138A"       "OR4F5"         "RP11.34P13.14" "OR4F29"       
[5] "RP5.857K21.2"  "OR4F16"

[1] 4476

In [15]:
non0_pseudocells = rowSums(df1>0)
head(non0_pseudocells)

MIR1302.10      FAM138A        OR4F5 RP11.34P13.7 RP11.34P13.8   AL627309.1 
          56            0            0          287           15          834

In [16]:
0.1*ncol(df1)

[1] 835.2

In [17]:
# remove genes in less 10% of pseudocells
low_genes = names(non0_pseudocells[non0_pseudocells<0.1*ncol(df1)])
head(low_genes)
length(low_genes)

[1] "MIR1302.10"   "FAM138A"      "OR4F5"        "RP11.34P13.7" "RP11.34P13.8"
[6] "AL627309.1"

[1] 19035

In [18]:
genes = rownames(df1)
length(genes)
head(genes)

[1] 32738

[1] "MIR1302.10"   "FAM138A"      "OR4F5"        "RP11.34P13.7" "RP11.34P13.8"
[6] "AL627309.1"

In [19]:
all_genes_df = df1

In [20]:
df1_new = df1[!(rownames(df1) %in% unique(c(zero_genes, low_genes))),]
nrow(df1_new)

[1] 13703

In [21]:
df1 = df1_new

In [22]:
genes = rownames(df1)
length(genes)
head(genes)

[1] 13703

[1] "AP006222.2"    "RP11.206L10.3" "RP11.206L10.9" "LINC00115"    
[5] "FAM41C"        "RP11.54O7.1"

In [23]:
cluster = 11

In [24]:
gene = genes[1]

In [25]:
head(as.numeric(df1[gene,]))
head(cluster_summaries[,cluster+1])

[1] 2.9360807 0.9526524 4.8315320 3.0357327 3.0441222 0.0000000

[1] -0.5376197 -0.6379603 -0.4618055 -0.5735192 -0.5656740 -0.6673057

In [26]:
cor(as.numeric(df1[gene,]), cluster_summaries[,cluster+1])

[1] 0.2794139

In [ ]:
gene_list = c()
for (gene in genes){
    gene_list = c(gene_list, cor(as.numeric(df1[gene,]), cluster_summaries[,cluster+1]))
}
names(gene_list) = genes
gene_list = gene_list[order(gene_list, decreasing=T)]

In [ ]:
hist(gene_list)

In [ ]:
genes_11_pos = gene_list[gene_list>0.5]
genes_11_neg = gene_list[gene_list<(-0.5)]
genes_11_neg = genes_11_neg[order(genes_11_neg)]

In [22]:
mat = matrix(0,nrow = length(genes), ncol = ncol(cluster_summaries))
rownames(mat) = genes
colnames(mat) = colnames(cluster_summaries)

In [23]:
nrow(mat)
ncol(mat)

[1] 13703

[1] 12

In [ ]:
for (cluster in 0:11){
    for (gene in genes){
        mat[gene,cluster+1] = cor(as.numeric(df1[gene,]), cluster_summaries[,cluster+1])
    }
}

In [ ]:
df_mat = as.data.frame(mat)
head(df_mat)
write.csv(df_mat, "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/May2021/genetic_effect/gene_expression_correlation_cluster_summaries.csv")

In [ ]:
for (i in 1:ncol(df_mat)){
    print(hist(df_mat[,i]))
}